# Goal: We want to have the LLM refine the current timebox json why issueing patches to the current json instead of creating a new one.
In the first iteration the LLM generates the timebox json from scratch.
For each iteration, the draft_schedule is validated, if there in a validation error this is returned to the LLM and the LLM is asked to fix the error.
The LLM then issues a patch to the current draft, this patch is applied deterministically and validated, if passes the current draft is shown to the user.
If the user wants to change the draft, the LLM is giving the instructions and the current draft, ot however important to note that we also want to include previous instructions from the user.

In [1]:
from pydantic import BaseModel, Field, model_validator
from datetime import date as datetime_date, timedelta, datetime
from typing import List, TYPE_CHECKING, Optional
from fateforger.agents.schedular.models import CalendarEvent

# Make sure CalendarEvent is defined somewhere above or below this cell
# TODO: make sure to inject the current date and time in the prompt so it can know when to set the date for the timebox
# TODO: make sure we use calc and anchor_prev
class Timebox(BaseModel):
    events: List[CalendarEvent] = Field(default_factory=list)
    date: Optional[datetime_date] = Field(default_factory=lambda: datetime_date.today() + timedelta(days=1),description="Date we are planning for, defaults to tomorrow")  # Default to tomorrow
    timezone: Optional[str] = Field(default="UTC", description="Timezone for the timebox, defaults to UTC")
    
    @model_validator(mode='before')
    def schedule_and_validate(cls, values):
        # Todo: check for non last bg event
        planning_date = values["date"]
        events= values["events"]

        # 1) Forward pass (default flex-forward)
        last_dt: datetime | None = None
        for ev in events:
            # compute two-of-three cases
            if ev.start and ev.duration and ev.end is None:
                ev.end = (datetime.combine(planning_date, ev.start) + ev.duration).time()
            elif ev.end and ev.duration and ev.start is None:
                ev.start = (datetime.combine(planning_date, ev.end) - ev.duration).time()
            elif ev.start and ev.end and ev.duration is None:
                ev.duration = datetime.combine(planning_date, ev.end) - datetime.combine(planning_date, ev.start)

            # flex-forward scheduling
            if ev.start is None and ev.end is None and not ev.anchor_prev:
                if last_dt is None:
                    raise ValueError(f"{ev.id}: needs start or duration")
                ev.start = last_dt.time()
                ev.end   = (last_dt + ev.duration).time()

            last_dt = datetime.combine(planning_date, ev.end)

        # 2) Backward pass (anchor_prev events)
        next_dt: datetime | None = None
        for ev in reversed(events):
            if ev.anchor_prev and ev.start is None and ev.end is None:
                if next_dt is None:
                    raise ValueError(f"{ev.id}: needs end or duration")
                ev.end   = next_dt.time()
                ev.start = (next_dt - ev.duration).time()
            next_dt = datetime.combine(planning_date, ev.start)

        # 3) Overlap check
        for a, b in zip(events, events[1:]):
            dt_a_end = datetime.combine(planning_date, a.end)
            dt_b_start = datetime.combine(planning_date, b.start)
            if dt_a_end > dt_b_start:
                raise ValueError(f"Overlap: {a.id} → {b.id}")

        # 4) Ensure last event’s start is within the planning day
        last_event = events[-1]
        dt_last_start = datetime.combine(planning_date, last_event.start)
        if dt_last_start.date() != planning_date:
            raise ValueError(
                f"{last_event.id}: start {dt_last_start} is not on {planning_date}"
            )

        return values

  <!-- Event Type Definitions -->
  <EventTypes>
    <Type id="M">Stakeholder meetings (fixed time)</Type>
    <Type id="C">Commute/travel</Type>
    <Type id="DW">Deep Work (≥90min focus)</Type>
    <Type id="SW">Shallow Work (admin/routines)</Type>
    <Type id="H">Habits (gym, mindfulness)</Type>
    <Type id="R">Recovery (meals, breaks)</Type>
    <Type id="BU">Buffer (overrun protection)</Type>
    <Type id="BG">Background tasks (can overlap)</Type>
    <Type id="PR">Planning/Review sessions</Type>
  </EventTypes>

  <!-- Output Specification -->
  <Output format="json">
    [{
      "event_type": "DW|M|SW|H|...",
      "summary": "Specific task name",
      "description": "Task details if needed",
      "start_time?": " (HH:MM, only if fixed)",
      "end_time?": " (HH:MM, only if fixed)",
      "duration?": "PTnHnM",
      "location": "Optional location details"
    }]

In [5]:
schema
# TODO: duration should be included in the event schema

{'$defs': {'CalendarEvent': {'properties': {'event_type': {'oneOf': [{'const': 'M',
       'description': 'stakeholder-driven appointments',
       'title': 'MEETING',
       'type': 'string'},
      {'const': 'C',
       'description': 'travel/transit',
       'title': 'COMMUTE',
       'type': 'string'},
      {'const': 'DW',
       'description': 'high-focus work (≥90 min)',
       'title': 'DEEP_WORK',
       'type': 'string'},
      {'const': 'SW',
       'description': 'routine/admin tasks',
       'title': 'SHALLOW_WORK',
       'type': 'string'},
      {'const': 'PR',
       'description': 'planning & review (system deep-clean)',
       'title': 'PLAN_REVIEW',
       'type': 'string'},
      {'const': 'H',
       'description': 'recurring routines & rituals',
       'title': 'HABIT',
       'type': 'string'},
      {'const': 'R',
       'description': 'meals, sleep & rest',
       'title': 'REGENERATION',
       'type': 'string'},
      {'const': 'BU',
       'description': 'bu

In [2]:
import pprint
from typing import List
from pydantic import TypeAdapter
from fateforger.agents.schedular.models.core import LLMJsonSchema

schema = TypeAdapter(Timebox).json_schema(
    schema_generator=LLMJsonSchema,
    mode="validation",
    # by_alias=True,
)
pprint.pprint(schema)

{'$defs': {'CalendarEvent': {'properties': {'anchor_prev': {'default': True,
                                                            'description': 'When '
                                                                           'both '
                                                                           'start '
                                                                           'and '
                                                                           'end '
                                                                           'are '
                                                                           'omitted: '
                                                                           'True '
                                                                           '→ '
                                                                           'start '
                                                                           'at '
            

In [ ]:
timebox_json = '''


'''

In [15]:
planned_events

AttributeError: 'str' object has no attribute 'color_id'

In [ ]:
# add to calendar

In [12]:
from pydantic import TypeAdapter

planned_events = TypeAdapter(List[CalendarEvent]).validate_json(timebox_json)
# timebox = Timebox(events=planned_events)


In [ ]:
from trustcall import create_extractor
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-5",
    temperature=0.8,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # api_key="...",
    # base_url="...",
    # organization="...",
    # other params...
)

calendar_agent = create_extractor(
    llm,                
    tools=[Timebox],          
    tool_choice="Timebox",  
    enable_inserts=True,
    enable_deletes=True           
)


result = calendar_agent.invoke({
    "messages": "Please move the team sync from 10 AM to 11 AM and shift all downstream events accordingly.",
    "existing": current_timebox_json
})

# 3. Under the hood:
#    a. The LLM is prompted to call the `validate_calendar` tool
#    b. If `existing` fails validation, trustcall asks for a JSON‐Patch diff
#    c. It applies that patch with python‐json‐patch…


NameError: name 'llm' is not defined

# we need to fetch the planned current state of the schedule from google calendar, turn it into a timebox json and provide it to the LLM as current draft. then the LLM can issue patches to it, checks for conflicts. if 

In [1]:
from datetime import date, time, timedelta
from pydantic import BaseModel, Field, model_validator
from trustcall import create_extractor
from langchain_openai import ChatOpenAI
from fateforger.agents.schedular.models import CalendarEvent

class Timebox(BaseModel):
    events: list[CalendarEvent] = Field(default_factory=list)
    date: date = Field(default_factory=lambda: date.today() + timedelta(days=1))
    timezone: str = Field(default="UTC")

    @model_validator(mode="before")
    def schedule_and_validate(cls, values):
        # ...same scheduling/overlap checks as in the notebook...
        return values

llm = ChatOpenAI(model="gpt-5", temperature=0.8)

# Whatever your current plan is; keep it as a plain dict keyed by the schema name
current_timebox = Timebox(
    date=date.today(),
    timezone="Europe/Amsterdam",
    events=[
        CalendarEvent(
            event_type="M",
            summary="Team sync",
            start_time=time(10, 0),
            end_time=time(10, 30),
            anchor_prev=False,
        ),
        CalendarEvent(
            event_type="DW",
            summary="Deep work",
            duration=timedelta(hours=2),
        ),
    ],
).model_dump(mode="json")

calendar_agent = create_extractor(
    llm,
    tools=[Timebox],
    tool_choice="Timebox",
    enable_updates=True,   # turn on PatchDoc loop
    enable_inserts=True,   # let the model add new events if needed
    enable_deletes=True,   # let the model remove events
)

result = calendar_agent.invoke(
    {
        "messages": "Please move the team sync from 10 AM to 11 AM and shift all downstream events accordingly.",
        "existing": {"Timebox": current_timebox},  # key must match the model name
    }
)

patched_timebox: Timebox = result["responses"][0]
patched_json = patched_timebox.model_dump(mode="json")

PydanticUserError: Error when building FieldInfo from annotated attribute. Make sure you don't have any field name clashing with a type annotation.

For further information visit https://errors.pydantic.dev/2.11/u/unevaluable-type-annotation